In [1]:
import fitbit
import requests
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import threading
import webbrowser

from oauthlib.oauth2 import BackendApplicationClient
from requests_oauthlib import OAuth2Session
from requests.auth import HTTPBasicAuth
from IPython.display import display, HTML

## Establish Connection to Fitbit API

In [2]:
CLIENT_ID = '22D937'
CLIENT_SECRET = 'ab29052c92affb67c9086e8ab6ed3158'
redirect_uri = 'http://127.0.0.1:8080/'
fitbit_date_format = '%Y-%m-%d'

In [3]:
scope = ['activity','heartrate', 'location', 'nutrition', 'profile', 'settings', 'sleep', 'social', 'weight']
auth  = HTTPBasicAuth(CLIENT_ID, CLIENT_SECRET)
oauth = OAuth2Session(client_id=CLIENT_ID,redirect_uri=redirect_uri,scope=scope)
auth_url, _ = oauth.authorization_url('https://www.fitbit.com/oauth2/authorize')

In [4]:
html = HTML(f"<a href='{auth_url}' target='_blank'>Click here and then copy the 'code' parameter in the URL</a>")
display(html)

In [5]:
code = input()

292f158b47d21bba973e7130de5815cd1b3a2543


In [6]:
token = oauth.fetch_token(token_url='https://api.fitbit.com/oauth2/token',username=CLIENT_ID,password=CLIENT_SECRET, code=code)

In [7]:
auth2_client = fitbit.Fitbit(CLIENT_ID, CLIENT_SECRET, oauth2=True, access_token=token['access_token'], refresh_token=token['refresh_token'])

In [8]:
auth2_client.API_VERSION = 1.2 #this has to be manually done because the python library defaults to ver 1.

## Build Sleep Data

In [ ]:
def fetch_data(resource, keys):
    date_list = []
    first_of_year = datetime.datetime.strptime('2018-01-01', fitbit_date_format)
    start_date = first_of_year
    for i in range(3):
        # FitBit only allows pulls of 100 days of data at a time. 
        hunnid_days_later = start_date + datetime.timedelta(days=100)
        records = auth2_client.time_series(resource=resource
                 , base_date=start_date.strftime(fitbit_date_format)
                 , end_date=hunnid_days_later.strftime(fitbit_date_format))
        
        lists = [][]
        
        for record in records[resource]:
            date_list.append(record['dateOfSleep'])
            startTime_list.append(record['startTime'])
            minutesAsleep_list.append(record['minutesAsleep'])
        
    start_date = hunnid_days_later + datetime.timedelta(days=1)

In [9]:
date_list = []
startTime_list = []
minutesAsleep_list = []
first_of_year = datetime.datetime.strptime('2018-01-01', fitbit_date_format)
start_date = first_of_year
for i in range(3):
    # FitBit only allows pulls of 100 days of data at a time. 
    hunnid_days_later = start_date + datetime.timedelta(days=100)
    sleep_records = auth2_client.time_series(resource='sleep'
             , base_date=start_date.strftime(fitbit_date_format)
             , end_date=hunnid_days_later.strftime(fitbit_date_format))
    for record in sleep_records['sleep']:
        date_list.append(record['dateOfSleep'])
        startTime_list.append(record['startTime'])
        minutesAsleep_list.append(record['minutesAsleep'])
        
    start_date = hunnid_days_later + datetime.timedelta(days=1)

In [10]:
df_sleep = pd.DataFrame({'StartTime':startTime_list, 'MinutesAsleep':minutesAsleep_list}, index=date_list)
df_sleep = df_sleep[['StartTime', 'MinutesAsleep']]

In [11]:
df_sleep = df_sleep.groupby(df_sleep.index).agg({'MinutesAsleep':sum, 'StartTime':'first'}).sort_index()

## Build Step Data

In [13]:
steps_list = []
date_list = []
first_of_year = datetime.datetime.strptime('2018-01-01', fitbit_date_format)
start_date = first_of_year
for i in range(3):
    # FitBit only allows pulls of 100 days of data at a time. 
    hunnid_days_later = start_date + datetime.timedelta(days=100)
    step_records = auth2_client.time_series(resource='activities/steps'
             , base_date=start_date.strftime(fitbit_date_format)
             , end_date=hunnid_days_later.strftime(fitbit_date_format))
    for record in step_records['activities-steps']:
        date_list.append(record['dateTime'])
        steps_list.append(record['value'])
        
    start_date = hunnid_days_later + datetime.timedelta(days=1)

In [14]:
df_steps = pd.DataFrame({'Steps': list(map(int,steps_list))}, index=date_list)

## Build Resting Heart Rate Data

In [31]:
hr_list = []
date_list = []
first_of_year = datetime.datetime.strptime('2018-01-01', fitbit_date_format)
start_date = first_of_year
for i in range(3):
    # FitBit only allows pulls of 100 days of data at a time. 
    hunnid_days_later = start_date + datetime.timedelta(days=100)
    hr_records = auth2_client.time_series(resource='activities/heart'
             , base_date=start_date.strftime(fitbit_date_format)
             , end_date=hunnid_days_later.strftime(fitbit_date_format))
    for record in hr_records['activities-heart']:
        if('restingHeartRate' in record['value']):
            date_list.append(record['dateTime'])
            hr_list.append(record['value']['restingHeartRate'])
        
    start_date = hunnid_days_later + datetime.timedelta(days=1)

In [17]:
df_heartrate = pd.DataFrame({'Heartrate': list(map(int,hr_list))}, index=date_list)

## Consolidate Data

In [19]:
df = pd.merge(df_sleep, df_steps, left_index=True, right_index=True, how="outer")

In [20]:
df = pd.merge(df, df_heartrate, left_index=True, right_index=True, how="outer")

In [22]:
df.to_csv("data")